In [ ]:
import os
gpu_device = 1    # número identificador del device puede ser: 0, 1, 2, o 3
os.environ["CUDA_VISIBLE_DEVICES"]=str(gpu_device)
from numba import cuda
cuda.select_device(0)

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset
import torch
import pandas as pd
import random
import math
from sklearn.metrics import classification_report, accuracy_score
from collections import defaultdict

In [ ]:
model_name="microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract"

In [ ]:
training_args = TrainingArguments(
    output_dir='./results',          # Carpeta para guardar el modelo
    num_train_epochs=5,             # Número de épocas
    per_device_train_batch_size=16,  # Tamaño del batch
    per_device_eval_batch_size=16,   # Tamaño del batch de validación
    learning_rate=0.01, 
    weight_decay=0.05,              # Decaimiento del peso
    logging_dir='./logs',           # Carpeta para los logs
    logging_steps=10,
    evaluation_strategy="epoch",    # Evaluar durante el entrenamiento
    save_strategy="epoch"           # Guardar el modelo al final de cada época
)

training_args_all_1= TrainingArguments(
    output_dir='./results',          # Carpeta para guardar el modelo
    num_train_epochs=3,             # Número de épocas
    per_device_train_batch_size=16,  # Tamaño del batch
    per_device_eval_batch_size=16,   # Tamaño del batch de validación
    learning_rate=0.00005, 
    weight_decay=0.05,              # Decaimiento del peso
    logging_dir='./logs',           # Carpeta para los logs
    logging_steps=10,
    evaluation_strategy="epoch",    # Evaluar durante el entrenamiento
    save_strategy="epoch"           # Guardar el modelo al final de cada época
)

training_args_all_2= TrainingArguments(
    output_dir='./results',          # Carpeta para guardar el modelo
    num_train_epochs=5,             # Número de épocas
    per_device_train_batch_size=16,  # Tamaño del batch
    per_device_eval_batch_size=16,   # Tamaño del batch de validación
    learning_rate=0.00003, 
    weight_decay=0.05,              # Decaimiento del peso
    logging_dir='./logs',           # Carpeta para los logs
    logging_steps=10,
    evaluation_strategy="epoch",    # Evaluar durante el entrenamiento
    save_strategy="epoch"           # Guardar el modelo al final de cada época
)

In [ ]:
%run ../utils/data.ipynb
%run ../utils/BERT-prior.ipynb

In [ ]:
dic={outputs[i]:0 for i in range(len(outputs))}
# print(targets)
# print(outputs)
for key,res in targets.items():
    print(key)
    print(res)
    # print(outputs[res])
    dic[outputs[res]]+=1
print(dic)
print(len(dic))

In [ ]:
len(outputs)

In [ ]:
predicted, tested=cross_validation(dataset_final,targets,dataset_final2,targets2)

In [ ]:
evaluate_per_question(predicted, tested, DICTIONARY)

In [ ]:
np.save("predicted_biomed__no_prior.npy", predicted)
np.save("tested_biomed__no_prior.npy", tested)